### Solar Panel Regular Expressions

CS 506 Spring 2018

Team: 

Jason Lu (jasonlu6@bu.edu)

Fang "Hugh" Qu (hughqu@bu.edu)

Zixin "Cindy" Ding (cindydxz@bu.edu)

In [46]:
# imports 
import json
import csv 
import pandas as pd
from pandas import DataFrame as Df

# clustering imports 
from sklearn.cluster import KMeans
from sklearn import mixture
import scipy.cluster.hierarchy as hierarchy
import sklearn.metrics as metrics
from sklearn.preprocessing import MinMaxScaler

# plotting imports 
import sklearn.cluster 
import matplotlib 
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

# Source Code:

In [47]:
# sources: 

# (json.dumps):
# https://stackoverflow.com/questions/19697846/how-to-convert-csv-file-to-multiline-json

# (avoid the UTF-8 encoding error):
# https://stackoverflow.com/questions/5552555/unicodedecodeerror-invalid-continuation-byte

# (TFDIF Vectorizer and Histogram Count Vectorizer for frequency of words in comments):
# https://stackoverflow.com/questions/36572221/how-to-find-ngram-frequency-of-a-column-in-a-pandas-dataframe

# creating the master database (create and read CSV)
# https://pythonspot.com/files-spreadsheets-csv/

# deal with error larger than field limit problem
# https://stackoverflow.com/questions/15063936/
# csv-error-field-larger-than-field-limit-131072

# avoid the "np.nan is an invalid document, expected byte or unicode string" Value Type error 
# https://stackoverflow.com/questions/39303912/
# tfidfvectorizer-in-scikit-learn-valueerror-np-nan-is-an-invalid-document

# counter object for the histogram frequency 
# https://stackoverflow.com/questions/22303554/words-frequency-using-pandas-and-matplotlib

# counting unique values in a list
# https://stackoverflow.com/questions/12282232/
# how-do-i-count-unique-values-inside-an-array-in-python

# plotting a frequency histogram
# https://stackoverflow.com/questions/45080698/
# make-frequency-histogram-from-list-with-tuple-elements

# selecting rows from dataframe using only a specified column value 
# (masking method)
# https://stackoverflow.com/questions/17071871/
# select-rows-from-a-dataframe-based-on-values-in-a-column-in-pandas

# filtering rows containing a string pattern / regex in pandas dataframe 
# https://stackoverflow.com/questions/27975069/
# how-to-filter-rows-containing-a-string-pattern-from-a-pandas-dataframe

# regular expression: extract a data type from string in pandas 
# https://stackoverflow.com/questions/35376387/extract-int-from-string-in-pandas

This code is used to parse the assessor CSV file and building permits CSV file in order to 
extract the solar capacity of each parcel in Boston based on zip codes. It is required to answer
one of our project questions: 

What communities are not benefitting from energy efficient systems? 

The steps to extracting
the solar capacity are as follows: 

1) We parse the assessor CSV file 

2) We parse the building permits CSV file 

3) Open the building permits csv file for solar panel text extraction

4) Get only the worktype of each csv file 

5) We then filter out only for 'SOL' worktype and print out the corresponding commments

6) Drop NaN values (for cleaning the data)

7) Get all of the comments into a list format 

8) We use regular expression to filter each comment from solar panel description

9) We then get only comments with kilowatts (which will contain information for solar capacity)

10) Saved data to a CSV file 

In [48]:
# parse the assessor CSV file (ast2018full.csv)
index = 0
pid_index = {}
master_list = {}
# attributes in master database 
attr = ['parcel_id','zipcode','land_use','avg_total','living_area',
        'heat_type','r_ac','comments','longitude','latitude']

with open('ast2018full.csv',encoding='latin-1') as fp:
    a_reader = csv.reader(fp, dialect='excel')
    n = next(a_reader,None)
    for row in a_reader:
        try:
            pid_index[row[0].lstrip("0")] 
        except KeyError:
            # create new data 
            master_list[index] = {}
            master_list[index] = {attr[0]: row[0],attr[1]: row[7],
                              attr[2]:row[9],attr[3]:row[18],
                              attr[4]:row[24],attr[5]:row[41],
                              attr[6]:row[42]}
            pid_index[row[0].lstrip("0")] = index
            index += 1
        else:
            # overwrite existing data 
            p_index = pid_index[row[0].lstrip("0")]
            master_list[p_index] = {attr[0]: row[0],attr[1]: row[7],
                              attr[2]:row[9],attr[3]:row[18],
                              attr[4]:row[24],attr[5]:row[41],
                              attr[6]:row[42]}
fp.close()

In [49]:
# parse the buildingpermits.csv file 
with open('buildingpermits.csv',newline='',encoding='latin-1') as assess_file:
    assess_reader = csv.reader(assess_file, dialect='excel')
    n = next(assess_reader,None)
    for row in assess_reader:
        try:
            pid_index[row[19].lstrip("0")] 
        except KeyError:
            # drop the data if parcel id not found in 
            # assessor database 
            pass
        else:
            # append information from building database to master database 
            index = pid_index[row[19].lstrip("0")]
            try:
                master_list[index][attr[7]]
            except KeyError:
                # if comment does not exist, add new comment 
                master_list[index][attr[7]] = row[4].replace('\t',' ').replace('\r',' ')
            else:
                # if comment exists, append comments 
                master_list[index][attr[7]] += " " + row[4].replace('\t',' ').replace('\r',' ')
            if row[20] != '':
                # get latitude and longitude from the location
                loc = row[20].lstrip('(').rstrip(')').split(',')
                master_list[index][attr[8]] = loc[0].replace(' ','')
                master_list[index][attr[9]] = loc[1].replace(' ','')
assess_file.close()

In [50]:
# import for dataframe and plots
import matplotlib.pyplot as plt 
import pandas as pd 
from pandas import DataFrame as Df

In [51]:
# filter out for only 'SOL' and print out the comments corresponding 
work = pd.read_csv('buildingpermits.csv',encoding='latin-1')
# work 

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [52]:
# avoid reading in NaN values 
filter_work = work[work['WORKTYPE'].notnull()]
filter_work = filter_work[filter_work['Comments'].notnull()]
# filter_work.head(10)

In [53]:
# reading the columns with only 'SOL' from worktype and corresponding comments 
# print out the capacity of the solar panels / energy systems 

# getting only the worktype and comments 
string_mask = filter_work[['WORKTYPE','Comments']]
string_mask

# filtering the worktype for 'SOL' only 

# regex expression of str.contains()
solar_mask = filter_work[filter_work['WORKTYPE'].str.contains("SOL*")]
# solar_mask

In [54]:
# imports 
# regular expression import 
import re
# import pandas 
import pandas as pd 

In [55]:
# get all of the comments into a list format 
df = solar_mask['Comments']
# df

solar_filter = filter_work[filter_work['Comments'].str.contains("solar*")]
# solar_filter

In [56]:
# extraction step for all float values 
extract = solar_filter['Comments'].str.extract('(\d+)').astype(float)
# filter out the NaNs 
extract_filter = solar_filter['Comments'].dropna()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  


In [57]:
import re

#get only comments with kilowatts 

# only decimal values for the kilowatts 
# drop NaN values and empty lists 
kw_extract = extract_filter.str.findall('\d+\.\d+').dropna()
kw_extract

109             [9.6]
414       [31.2, 7.5]
2617               []
2634               []
2667          [27.69]
2897           [67.2]
2901               []
2925               []
2944         [195.84]
3056               []
3066               []
3118               []
3145          [24.48]
3346               []
3365               []
3366               []
3375               []
3376               []
3377               []
3378               []
3379               []
3380               []
3381               []
3382               []
3383               []
3384               []
3385               []
3388               []
3389               []
3390               []
             ...     
354925             []
354939         [9.57]
355071         [4.02]
355087        [4.060]
355256         [6.38]
355258         [2.23]
355305         [3.77]
355355         [3.92]
355357         [6.44]
355373         [2.95]
355402        [11.21]
355408          [8.4]
355441        [3.245]
355458         [6.09]
355469    

In [58]:
frame = pd.Series.to_frame(kw_extract)
frame.to_csv("kw.csv")